In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np

In [2]:
train = pd.read_csv('./자료/와인/train.csv')
test = pd.read_csv('./자료/와인/test.csv')

# Scailing
scaler = MinMaxScaler()
scaler.fit(train[['fixed acidity']])
train['Scaled fixed acidity'] = scaler.transform(train[['fixed acidity']])
test['Scaled fixed acidity'] = scaler.transform(test[['fixed acidity']])

# Encoding
encoder = OneHotEncoder()
encoder.fit(train[['type']])
onehot = encoder.transform(train[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
train = pd.concat([train, onehot], axis = 1)
train = train.drop(columns = ['type'])

onehot = encoder.transform(test[['type']])
onehot = onehot.toarray()
onehot = pd.DataFrame(onehot)
onehot.columns = encoder.get_feature_names()
test = pd.concat([test, onehot], axis = 1)
test = test.drop(columns = ['type'])

test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Scaled fixed acidity,x0_red,x0_white
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0.429752,0.0,1.0
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,0.785124,1.0,0.0
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0.223140,0.0,1.0
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0.280992,0.0,1.0
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0.247934,0.0,1.0


In [3]:
X = train.drop(columns = ['index','quality'])
Y = train['quality']

In [4]:
kf = KFold(n_splits = 5, shuffle = True, random_state = 0)

In [5]:
model = RandomForestClassifier(random_state = 0)
valid_scores = []
test_predictions = []

In [6]:
for train_idx, valid_idx in kf.split(X,Y) : 
  X_tr = X.iloc[train_idx]
  Y_tr = Y.iloc[train_idx]

  X_val = X.iloc[valid_idx]
  Y_val = Y.iloc[valid_idx]

  model.fit(X_tr, Y_tr)
  
  valid_prediction = model.predict(X_val)
  score = accuracy_score(Y_val, valid_prediction)
  valid_scores.append(score)
  print(score)
print('평균 점수 : ', np.mean(valid_scores))

0.69
0.6663636363636364
0.6569608735213831
0.6760691537761602
0.6715195632393085
평균 점수 :  0.6721826453800976


In [8]:
for train_idx, valid_idx in kf.split(X,Y) : 
  X_tr = X.iloc[train_idx]
  Y_tr = Y.iloc[train_idx]

  X_val = X.iloc[valid_idx]
  Y_val = Y.iloc[valid_idx]

  model.fit(X_tr, Y_tr)

  test_prediction = model.predict(test.drop(columns = ['index']))
  test_predictions.append(test_prediction)

In [9]:
test_predictions = pd.DataFrame(test_predictions)
test_predictions

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,5,6,6,5,6,6,6,6,6,7,...,6,6,6,5,7,6,6,5,6,6
1,5,6,6,5,6,6,6,6,6,7,...,6,6,6,6,7,6,6,5,6,6
2,6,6,6,5,6,6,6,6,6,7,...,6,6,6,6,7,5,6,5,6,6
3,5,6,6,6,6,6,5,6,6,6,...,6,6,6,6,7,6,6,5,6,6
4,5,6,6,5,7,6,5,6,6,7,...,6,6,6,6,6,6,5,5,6,7


In [10]:
test_prediction = test_predictions.mode()
test_prediction = test_predictions.values[0]
test_prediction

array([5, 6, 6, 5, 6, 6, 6, 6, 6, 7, 6, 5, 7, 6, 5, 5, 6, 7, 5, 6, 6, 6,
       5, 7, 6, 5, 5, 5, 5, 5, 7, 6, 5, 5, 6, 6, 5, 5, 6, 6, 6, 5, 5, 5,
       6, 6, 5, 6, 4, 6, 5, 5, 6, 4, 5, 6, 6, 5, 5, 6, 5, 6, 6, 6, 6, 6,
       5, 5, 6, 5, 5, 5, 5, 5, 6, 5, 6, 6, 6, 7, 6, 6, 5, 5, 6, 6, 5, 5,
       6, 6, 6, 6, 6, 6, 6, 7, 6, 6, 6, 6, 5, 6, 5, 5, 6, 6, 6, 6, 7, 5,
       7, 6, 6, 5, 5, 6, 6, 7, 7, 5, 6, 5, 6, 7, 5, 5, 6, 6, 6, 7, 5, 5,
       6, 5, 5, 6, 6, 7, 5, 5, 5, 6, 5, 5, 5, 5, 7, 6, 7, 6, 5, 8, 6, 5,
       6, 5, 6, 5, 6, 5, 5, 6, 6, 7, 6, 6, 6, 6, 5, 6, 5, 5, 6, 5, 6, 6,
       6, 5, 6, 6, 6, 6, 7, 7, 5, 6, 6, 6, 5, 6, 6, 6, 6, 6, 6, 6, 6, 5,
       5, 5, 6, 5, 6, 5, 5, 5, 6, 6, 6, 6, 7, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 7, 6, 6, 5, 6, 5, 6, 7, 7, 6, 6, 6, 5, 6, 6, 7, 6, 6,
       6, 6, 7, 6, 5, 6, 6, 6, 5, 6, 7, 7, 6, 6, 6, 7, 7, 6, 7, 5, 5, 7,
       5, 5, 6, 5, 6, 6, 6, 5, 5, 5, 6, 5, 7, 7, 5, 6, 6, 6, 6, 5, 7, 7,
       6, 6, 6, 6, 5, 5, 5, 6, 5, 7, 8, 6, 5, 7, 6,

In [11]:
sample_submission = pd.read_csv('./자료/와인/sample_submission.csv')
sample_submission['quality'] = test_prediction
sample_submission.to_csv('./자료/와인/submission_KFOLD.csv', index = False)